In [1]:
import pandas as pd
import data

filepath = "../data/papers/urban-agriculture.csv"
df = pd.read_csv(filepath)
df = data.load_scopus(df)

2020-06-05 21:12:03,789 - INFO - Renaming and selecting columns ...
2020-06-05 21:12:09,473 - INFO - Formatting author names ...
2020-06-05 21:12:09,477 - INFO - Disambiguating author names ...
2020-06-05 21:12:09,540 - INFO - Removing part of titles in foreing languages ...
2020-06-05 21:12:09,543 - INFO - Fusioning author and index keywords ...
2020-06-05 21:12:09,556 - INFO - NumExpr defaulting to 8 threads.
2020-06-05 21:12:09,571 - INFO - Extracting countries from affiliations ...
2020-06-05 21:12:15,858 - INFO - Extracting institutions from affiliations ...
2020-06-05 21:12:15,871 - INFO - Extracting country of 1st author ...
2020-06-05 21:12:15,877 - INFO - Extracting affiliation of 1st author ...
2020-06-05 21:12:15,881 - INFO - Counting number of authors ...


In [2]:
import co_occurrence

co_occurrence.app(df)

AppLayout(children=(HTML(value="<h1>Co-occurrence Analysis</h1><hr style='height:2px;border-width:0;color:gray…

In [ ]:
import correlation

correlation.app(df)

In [ ]:
data.coverage(df)

### Keywords

### FindString Dialog Box pag. 69

### Cleaning a list pag. 102

### Fuzzy match editor pag.126

### Stemming And / Stemming Or

### APPS

In [ ]:
import techminer.by_year as year
year.app(df)

In [ ]:
import techminer.by_term as term
term.app(df)

In [ ]:
import techminer.by_term_per_year as term_year
term_year.app(df)

In [ ]:
import techminer.co_occurrence as co_occurrence
co_occurrence.app(df)

In [ ]:
import techminer.correlation as correlation
correlation.app(df)

In [ ]:
import ipywidgets as widgets
from ipywidgets import Layout, AppLayout

In [ ]:

COLUMNS = [
    "Author Keywords",
    "Authors",
    "Countries",
    "Country 1st",
    "Document type",
    "Index Keywords",
    "Institution 1st",
    "Institutions",
    "Keywords",
    "Source title",
]

In [ ]:
FIGSIZE = (14, 10.0) ## Co-occurrente-analysis  <---- no tiene figura
FIGSIZE = (14, 9.0)

# NUEVA INTERFAZ

In [ ]:
from techminer.summary_by_term_per_year import documents_by_term_per_year, citations_by_term_per_year

In [ ]:
import techminer.plots as plt
from techminer.plots import COLORMAPS
from techminer.summary_by_term import citations_by_term, documents_by_term

WIDGET_WIDTH = "200px"
LEFT_PANEL_HEIGHT = "588px"
RIGHT_PANEL_WIDTH = "870px"
FIGSIZE = (14, 10.0)
PANE_HEIGHTS = ["80px", "650px", 0]

COLUMNS = [
    "Author Keywords",
    "Authors",
    "Countries",
    "Country 1st",
    "Document type",
    "Index Keywords",
    "Institution 1st",
    "Institutions",
    "Keywords",
    "Source title",
]



In [ ]:
import co_occurrence

In [ ]:
from ipywidgets import interact, interactive, fixed, interact_manual
interact(slow_function,i=widgets.FloatSlider(min=1e5, max=1e7, step=1e5, continuous_update=True));

In [ ]:
def slow_function(i):
    print(int(i),list(x for x in range(int(i)) if
                str(x)==str(x)[::-1] and
                str(x**2)==str(x**2)[::-1]))
    return

In [ ]:
x = "hola"
x[:-1]

In [ ]:
import pandas as pd
x = pd.DataFrame({'a': [1, 2, 3, 4, 5], 'b': [1, 2, 3, 4, 5]})

pd.Series(x.values.ravel()).value_counts()

In [ ]:
pd.melt(x)

In [ ]:
x.values.shape

In [ ]:
print('hola')

In [ ]:
m = "abcdefg [1000]"
m[m.find('[')+1:m.find(']')]


In [ ]:
[1, 2, 3] == (1, 2, 3)

In [ ]:
import networkx as nx
G = nx.petersen_graph()
nx.draw(G, with_labels=True, font_weight='bold')

In [ ]:
nx.draw_shell(G, nlist=[range(5, 10), range(5)], with_labels=True, font_weight='bold')

In [ ]:
nx.draw_circular(G, nlist=[range(5, 10), range(5)], with_labels=True, font_weight='bold')

In [ ]:
nx.draw_kamada_kawai(G, nlist=[range(5, 10), range(5)], with_labels=True, font_weight='bold')

In [ ]:
nx.draw_spectral(G, nlist=[range(5, 10), range(5)], with_labels=True, font_weight='bold')

In [ ]:
nx.draw_spring(G, nlist=[range(5, 10), range(5)], with_labels=True, font_weight='bold')

In [ ]:
nx.draw_shell(G, nlist=[range(5, 10), range(5)], with_labels=True, font_weight='bold')

In [ ]:
[12]*5